In [105]:
import pandas as pd
import numpy as np
import codecs
import os
filename = "input/lvl3-0.inp"
output_file = "output.txt"



In [128]:
images = []
line_cnt = 0
start_ts = 0
end_ts = 0

with codecs.open(filename,"r", "utf-8") as f:
    temp = f.read().splitlines()
    stats = temp[0].split(" ")
    start_ts = int(stats[0])
    end_ts = int(stats[1])
    line_cnt += 1
    for i in range(int(stats[2])):
        img_desc = temp[line_cnt].split(" ")
        tmp_img = np.empty((int(img_desc[1]), int(img_desc[2])))
        line_cnt+=1
        for x in range(int(img_desc[1])):
            tmp_img[x] = temp[line_cnt+x].split(" ")
        images.append([img_desc,tmp_img])
        line_cnt += int(img_desc[1]) 


        
occurence_dict = dict()
for img in images:
    img_arr = img[1]
    # remove empty rows
    img_arr = img_arr[(~np.all(img_arr == 0, axis=1))]
    # remove empty columns
    img_arr = img_arr[:, ~np.all(img_arr == 0, axis=0)]
    # set all postive to one
    img_arr[img_arr > 0] = 1
    if (img_arr > 0).any():
        if img_arr.tobytes() in occurence_dict:
            # asteroid already detected -> update last occurence
            asteroid = occurence_dict[img_arr.tobytes()]
            asteroid[1] = int(img[0][0])
            asteroid[2].append(int(img[0][0]))
            occurence_dict[img_arr.tobytes()] = asteroid
        else:
            # new asteroid
            occurence_dict[img_arr.tobytes()] = [int(img[0][0]), int(img[0][0]), [int(img[0][0])]]

occ_arr = []          
for _, occurenec_arr in occurence_dict.items():
    occ_arr.append(occurenec_arr)
#np_occ = np.array(occ_arr)
#with codecs.open(output_file,"w", "utf-8") as f:
#    for arr in np_occ[np_occ[:,0].argsort()]:
#        f.write(' '.join(map(str, arr)) + os.linesep)

In [146]:
candidate_lst = []
for arr in occ_arr:
    time_sq = arr[2]
    for start_idx, time_point in enumerate(time_sq):
        print("-----")
        print(candidate_lst)
        for candidate in candidate_lst:
            print(candidate)
            if len(candidate) > 1:
                length = candidate[1] - candidate[0] # current length
                print(candidate, length, time_point - candidate[-1], "before")
                if length == time_point - candidate[-1]: # matching length
                    print(candidate, time_point, "in")
                    candidate.append(time_point)
                elif length < time_point - candidate[-1] and len(candidate) < 4:                   
                    # candidate invalid
                    print(time_point, candidate)
                    candidate_lst.remove(candidate)
            elif len(candidate) == 1:
                # fetch spawning error
                if time_point - candidate[0] < candidate[0] - start_ts and candidate[0] != start_ts:
                    pass
                #    print(time_point, candidate)
                #    candidate_lst.remove(candidate)
                else:
                    candidate_lst.append([candidate[0], time_point])
            else:
                candidate.append(time_point)
        candidate_lst.append([time_point])
        
        
for candidate in candidate_lst:
    print(candidate, "xxx")
    if len(candidate) > 3:
        print(candidate)

-----
[]
-----
[[1]]
[1]
[1, 4]
[1, 4] 3 0 before
-----
[[1], [1, 4], [4]]
[1]
[1, 4]
[1, 4] 3 3 before
[1, 4] 7 in
[4]
[1, 7]
[1, 7] 6 0 before
[4, 7]
[4, 7] 3 0 before
-----
[[1], [1, 4, 7], [4], [1, 7], [4, 7], [7]]
[1]
[1, 4, 7]
[1, 4, 7] 3 1 before
[4]
[1, 7]
[1, 7] 6 1 before
[4, 7]
[4, 7] 3 1 before
[7]
[1, 8]
[1, 8] 7 0 before
[4, 8]
[4, 8] 4 0 before
-----
[[1], [1, 4, 7], [4], [1, 7], [4, 7], [7], [1, 8], [4, 8], [8]]
[1]
[1, 4, 7]
[1, 4, 7] 3 5 before
12 [1, 4, 7]
[1, 7]
[1, 7] 6 5 before
[4, 7]
[4, 7] 3 5 before
12 [4, 7]
[1, 8]
[1, 8] 7 4 before
[4, 8]
[4, 8] 4 4 before
[4, 8] 12 in
[8]
[1, 12]
[1, 12] 11 0 before
-----
[[1], [4], [1, 7], [7], [1, 8], [4, 8, 12], [8], [1, 12], [12]]
[1]
[4]
[1, 7]
[1, 7] 6 6 before
[1, 7] 13 in
[7]
[1, 8]
[1, 8] 7 5 before
[4, 8, 12]
[4, 8, 12] 4 1 before
[8]
[1, 12]
[1, 12] 11 1 before
[12]
[1, 13]
[1, 13] 12 0 before
[4, 13]
[4, 13] 9 0 before
[7, 13]
[7, 13] 6 0 before
-----
[[1], [4], [1, 7, 13], [7], [1, 8], [4, 8, 12], [8], [1, 12], 

In [130]:
start_ts, end_ts

(1, 19)

In [141]:
occ_arr[0][2]

[1, 4, 7, 8, 12, 13, 16, 19]